In [15]:
#creates connectivity matrix
import os
import sys
import pandas as pd
import numpy as np
import nibabel as nib
from nilearn import image, input_data, datasets
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Import your parameters
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'
sys.path.insert(0, curr_dir)
import ptoc_params as params

# Set up directories and parameters
study = 'ptoc'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
raw_dir = params.raw_dir
results_dir = '/user_data/csimmon2/git_repos/ptoc/results'

# Load subject information
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')
subjects_to_skip = []
subs = sub_info[(sub_info['group'] == 'control') & (~sub_info['sub'].isin(subjects_to_skip))]['sub'].tolist()

run_num = 3
runs = list(range(1, run_num + 1))

# Define the number of ROIs for the Schaefer atlas
n_rois = 200

def verify_standard_space(img):
    """Verify image is in 2mm standard space"""
    if img.shape[:3] != (91, 109, 91):
        logging.warning(f"Unexpected shape: {img.shape}")
        return False
    
    vox_size = np.sqrt(np.sum(img.affine[:3, :3] ** 2, axis=0))
    if not np.allclose(vox_size, [2., 2., 2.], atol=0.1):
        logging.warning(f"Unexpected voxel size: {vox_size}")
        return False
    
    return True

def create_connectivity_matrix(ss):
    """Create connectivity matrix using standard space data"""
    logging.info(f"Processing subject: {ss}")
    
    # Load Schaefer atlas (already in MNI 2mm space)
    atlas = datasets.fetch_atlas_schaefer_2018(n_rois=n_rois, yeo_networks=7, resolution_mm=2)
    atlas_img = atlas.maps
    
    all_runs_data = []
    
    for rn in runs:
        # Load standard space data
        run_path = f'{raw_dir}/{ss}/ses-01/derivatives/fsl/loc/run-0{rn}/1stLevel.feat/reg_standard/filtered_func_data_standard.nii.gz'
        if not os.path.exists(run_path):
            logging.warning(f'Standard space data not found: {run_path}')
            continue
        
        subject_img = nib.load(run_path)
        
        # Verify standard space
        if not verify_standard_space(subject_img):
            logging.warning(f"Data not in expected standard space for {ss} run-{rn}")
            continue
        
        # Extract time series using the atlas
        masker = NiftiLabelsMasker(
            labels_img=atlas_img,
            standardize=True,
            memory='nilearn_cache',
            verbose=0
        )
        
        time_series = masker.fit_transform(subject_img)
        all_runs_data.append(time_series)
    
    if not all_runs_data:
        logging.warning(f'No valid run data found for subject {ss}')
        return None
    
    # Concatenate runs
    full_time_series = np.concatenate(all_runs_data, axis=0)
    
    # Compute connectivity matrix
    correlation_measure = ConnectivityMeasure(kind='correlation')
    connectivity_matrix = correlation_measure.fit_transform([full_time_series])[0]
    
    return connectivity_matrix

def main():
    for ss in subs:
        connectivity_matrix = create_connectivity_matrix(ss)
        if connectivity_matrix is not None:
            # Save with modified filename to indicate standard space
            output_dir = f'{results_dir}/connectivity_matrices_{n_rois}_standard'
            os.makedirs(output_dir, exist_ok=True)
            np.save(f'{output_dir}/{ss}_connectivity_matrix_standard.npy', connectivity_matrix)
            logging.info(f'Saved standard space connectivity matrix for {ss}')

if __name__ == "__main__":
    main()

2024-11-08 16:50:45,902 - INFO - Processing subject: sub-025


/home/csimmon2/anaconda3/envs/fmri/lib/python3.9/site-packages/nilearn/maskers/nifti_labels_masker.py:617: UserWarning: Persisting input arguments took 1.02s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  region_signals, labels_ = self._cache(
/home/csimmon2/anaconda3/envs/fmri/lib/python3.9/site-packages/nilearn/maskers/nifti_labels_masker.py:617: UserWarning: Persisting input arguments took 1.00s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with

In [16]:
import os
import nibabel as nib
import numpy as np

def examine_registration(subject='sub-025', run='01'):
    """Examine all registration-related files and their properties"""
    
    feat_dir = f'{raw_dir}/{subject}/ses-01/derivatives/fsl/loc/run-{run}/1stLevel.feat'
    
    # Check original filtered_func_data
    print("\nOriginal filtered_func_data:")
    func_img = nib.load(f'{feat_dir}/filtered_func_data.nii.gz')
    print(f"Shape: {func_img.shape}")
    print(f"Affine:\n{func_img.affine}")
    
    # Check example_func image
    print("\nExample func:")
    ex_func = nib.load(f'{feat_dir}/reg/example_func.nii.gz')
    print(f"Shape: {ex_func.shape}")
    print(f"Affine:\n{ex_func.affine}")
    
    # Check standard reference
    print("\nStandard reference:")
    std = nib.load(f'{feat_dir}/reg/standard.nii.gz')
    print(f"Shape: {std.shape}")
    print(f"Affine:\n{std.affine}")
    
    # Print transformation matrices
    print("\nTransformation matrices:")
    matrices = [
        'example_func2highres.mat',
        'example_func2standard.mat',
        'highres2standard.mat'
    ]
    
    for mat in matrices:
        mat_path = f'{feat_dir}/reg/{mat}'
        if os.path.exists(mat_path):
            print(f"\n{mat}:")
            with open(mat_path, 'r') as f:
                print(f.read().strip())
        else:
            print(f"\n{mat} not found")

# Examine a successful case
examine_registration('sub-025', '01')


Original filtered_func_data:
Shape: (106, 106, 69, 184)
Affine:
[[-1.99940538e+00 -4.50236648e-02 -1.87405590e-02  1.09541924e+02]
 [-4.52896506e-02  1.99928153e+00  2.86750533e-02 -9.74145966e+01]
 [-1.80882998e-02 -2.90909018e-02  1.99970675e+00 -4.35973511e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Example func:
Shape: (106, 106, 69)
Affine:
[[-1.99940538e+00 -4.50236648e-02 -1.87405590e-02  1.09541924e+02]
 [-4.52896506e-02  1.99928153e+00  2.86750533e-02 -9.74145966e+01]
 [-1.80882998e-02 -2.90909018e-02  1.99970675e+00 -4.35973511e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Standard reference:
Shape: (176, 256, 256)
Affine:
[[ 9.99702632e-01 -2.03442201e-02 -1.33508909e-02 -8.24147644e+01]
 [ 2.25902833e-02  9.79854107e-01  1.98432773e-01 -1.30027405e+02]
 [ 9.04495362e-03 -1.98675618e-01  9.80023563e-01 -1.08843857e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

Transformation matrices:

exam